# Traveling Salesman Problem using Variants of Quantum Approximate Optimization Algorithm (QAOA), a small illustration of QiboOpt

## Traveling Salesman Problem

The **Traveling Salesman Problem (TSP)** is one of the most famous problems in combinatorial optimization. It can be stated as follows:

> Given a list of cities and the distances between each pair of cities, the goal is to find the **shortest possible route** that visits each city **exactly once** and returns to the starting city.

**Key Points:**

1. **Combinatorial Nature:** For $n$ cities, there are $(n-1)!$ possible routes (since the starting city can be fixed). This grows factorially, making TSP computationally hard for large $n$.  

2. **Applications:** TSP is not just theoretical, it appears in logistics, vehicle routing, circuit design, and even DNA sequencing.  

3. **Optimization Goal:** We aim to minimize the **total travel distance** (or cost) of the tour.



Here is a $3$-city example where we have to decide which direction to travel to minimize the cost. We consider the following distance matrices

$$
\mathbf{D} =
\begin{bmatrix}
0 & 0.9 & 0.8 \\
0.4 & 0 & 0.1 \\
0.5 & 0.7 & 0
\end{bmatrix}
$$

where the $(i,j)$ entry of the distance matrix indicates the travel distance from city $i$ to city $j$. Notice that the distance matrix does not need to  be symmetrical. Let's visualize the two possible directions.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

dist_matrix = np.array([
    [0, 0.9, 0.8],
    [0.4, 0, 0.1],
    [0.5, 0.7, 0]
])

In [ ]:
# Build directed graph
G = nx.DiGraph()
n = len(dist_matrix)
for i in range(n):
    for j in range(n):
        if dist_matrix[i, j] > 0:
            G.add_edge(i, j, weight=dist_matrix[i, j])

# Two tours
tour1 = [(0, 1), (1, 2), (2, 0)]
tour2 = [(0, 2), (2, 1), (1, 0)]

# Compute total lengths
tour1_length = sum(dist_matrix[i, j] for i, j in tour1)
tour2_length = sum(dist_matrix[i, j] for i, j in tour2)

pos = nx.circular_layout(G)  # nice circular layout

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# --- Tour 1 in green ---
ax = axes[0]
nx.draw_networkx_edges(G, pos, ax=ax, edge_color="lightgray", arrows=True,
                       connectionstyle="arc3,rad=0.1", arrowstyle='-|>', min_source_margin=15, min_target_margin=15)
nx.draw_networkx_edges(G, pos, edgelist=tour1, ax=ax,
                       edge_color="green", width=2, arrows=True,
                       connectionstyle="arc3,rad=0.1", arrowstyle='-|>', min_source_margin=15, min_target_margin=15)
nx.draw_networkx_nodes(G, pos, ax=ax, node_color="lightblue", node_size=800)
nx.draw_networkx_labels(G, pos, ax=ax, font_size=12, font_color="black")
edge_labels = {(i, j): f"{dist_matrix[i, j]:.1f}" for i, j in tour1}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax, font_size=10)
ax.set_title(f"Tour 1 (Green) — Total Length = {tour1_length:.2f}")

# --- Tour 2 in red ---
ax = axes[1]
nx.draw_networkx_edges(G, pos, ax=ax, edge_color="lightgray", arrows=True,
                       connectionstyle="arc3,rad=0.1", arrowstyle='-|>', min_source_margin=15, min_target_margin=15)
nx.draw_networkx_edges(G, pos, edgelist=tour2, ax=ax,
                       edge_color="red", width=2, arrows=True,
                       connectionstyle="arc3,rad=0.1", arrowstyle='-|>', min_source_margin=15, min_target_margin=15)
nx.draw_networkx_nodes(G, pos, ax=ax, node_color="lightblue", node_size=800)
nx.draw_networkx_labels(G, pos, ax=ax, font_size=12, font_color="black")
edge_labels = {(i, j): f"{dist_matrix[i, j]:.1f}" for i, j in tour2}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax, font_size=10)
ax.set_title(f"Tour 2 (Red) — Total Length = {tour2_length:.2f}")

plt.tight_layout()
plt.show()


We will prefer Tour 1 as the distance is smaller than Tour 2. 

In this tutorial, we will showcase a small instance of 3 cities problem and show that we can determine the right direction to travel. This is just a proof of concept of how to formulate TSP in a form that can be attempted using a quantum optimisation algorithm known as Quantum Approximate Optimization Algorithm (QAOA). 

## Encoding of the length of the tour as a Quadratic Unconstrained Binary Optimisation (QUBO) Problem

The length of a tour can be encoded as a Quadratic Unconstrained Binary Optimisation (QUBO) problem. A common encoding is as follows, consider an $n$-city problem, we prepare $n$ slots that will determine the order in which we visit the cities. 

We then use a one-hot encoding of $x_{ui}=1$ if city $u$ is assigned to slot $i$, otherwise we have $x_{ui}=0$ if city $u$ is not assigned to slot $i$. We let $$d_{u,v}$$ represent the distance.

Given an encoding of a valid tour, the length of the tour can be described as 

$$\sum_{i=0}^{n-1}  \sum_{u=0}^{n-1}\sum_{v=0}^{n-1}d_{u,v}x_{u,i}x_{v, i+1 \pmod{n}}.$$

It is summing up the the distance of cities that are visited consecutively.

Using the technique developed by Hadfield [4],  quantum alternating operator ansatz algorithm can be used to solve this TSP instance provided it begins with a feasible solution. Both this encoding and the technique used to ensure feasibility remains has been implemented in the TSP class that we can simply import using `from qiboopt.combinatorial.combinatorial import TSP` and we just have to provide the distance function to the function `TSP`. 

In [ ]:
from qiboopt.combinatorial.combinatorial import TSP
import numpy as np
from collections import defaultdict
from qibo import gates
from qibo.models import QAOA
from qibo.result import CircuitResult
from qibo.models.circuit import Circuit


def check_feasibility(config):
    """
    check if a configuration is feasible
    """
    m = int(np.sqrt(len(config)))
    for i in range(m):
        row_sum = 0
        for j in range(m):
            row_sum += int(config[m*i+j])
        if row_sum != 1:
                return False
    for j in range(m):
        col_sum = 0
        for i in range(m):
            col_sum += int(config[m*i+j])
        if col_sum != 1:
            return False
    return True
    
def convert_to_standard_Cauchy(config):
    """
    Given a binary string of n^2, we convert it to an order of 0, 1, ..., n-1
    """
    m = int(np.sqrt(len(config)))
    cauchy = [-1] * m  # Cauchy's notation for permutation, e.g. (1,2,0) or (2,0,1)
    for i in range(m):
        for j in range(m):
            if config[m * i + j] == '1':
                cauchy[j] = i  # citi i is in slot j
    for i in range(m):
        if cauchy[i] == 0:
            cauchy = cauchy[i:] + cauchy[:i]
            return tuple(cauchy)
            # now, the cauchy notation for permutation begins with 0


def evaluate_dist(cauchy):
    '''
    Given a permutation of 0 to n-1, we compute the distance of the tour

    '''
    m = len(cauchy)
    return sum(distance_matrix[cauchy[i]][cauchy[(i+1)%m]] for i in range(m))


def find_mode_from_result(freq_counter):
    # let's combine freq_counter here, first convert each key and sum up the frequency
    cauchy_dict = defaultdict(int)
    feasibility_counter = 0
    freq_counter_len = 0
    feasibility = 0
    for freq_key in freq_counter:
        freq_counter_len += freq_counter[freq_key]
        if check_feasibility(freq_key):
            standard_cauchy_key = convert_to_standard_Cauchy(freq_key)
            cauchy_dict[standard_cauchy_key] += freq_counter[freq_key]
            feasibility += freq_counter[freq_key]
    if feasibility == 0:
        print("warning, no solution is feasible")
        return None, None, 0
    max_key = max(cauchy_dict, key=cauchy_dict.get)
    feasibility /= freq_counter_len
    return max_key, evaluate_dist(max_key), feasibility, cauchy_dict
    

def qaoa_function_of_layer(layer, distance_matrix):
    '''
    This is a function to study the impact of the number of layers on QAOA,
    it takes in the number of layers and compute the distance of the mode
    of the histogram obtained from QAOA

    '''
    small_tsp = TSP(distance_matrix)
    obj_hamil, mixer = small_tsp.hamiltonians()
    qaoa = QAOA(obj_hamil, mixer=mixer)
    best_energy, final_parameters, extra = qaoa.minimize(initial_p=[0.1] * layer,
                                         initial_state=initial_state, method='BFGS')
    qaoa.set_parameters(final_parameters)
    quantum_state = qaoa.execute(initial_state)
    circuit = Circuit(9)
    circuit.add(gates.M(*range(9)))
    result = CircuitResult(quantum_state, circuit.measurements,
            small_tsp.backend, nshots=1000)
    freq_counter = result.frequencies()
    return find_mode_from_result(freq_counter) 

np.random.seed(42)
num_cities = 3
distance_matrix = dist_matrix
distance_matrix = distance_matrix.round(1)
small_tsp = TSP(distance_matrix)
initial_parameters = np.random.uniform(0, 1, 2)
initial_state = small_tsp.prepare_initial_state([i for i in range(num_cities)])
qaoa_function_of_layer(2, distance_matrix)

Another approach that we can use is the penalty method where we have to set a penalty coefficient. 

The constraints can be described mathematically as

$$\sum_{i} x_{ui}=1, \forall u \in \{0, \ldots, n-1\}$$

and 

$$\sum_{u} x_{ui}=1, \forall i \in \{0, \ldots, n-1\}$$

which means each city is allocated to exactly one slot and each slot is allocated to exactly one city. 

Using the penalty method, the modified objective function is as follows:

$$\sum_{i=0}^{n-1}\sum_{u=0}^{n-1}\sum_{v=0}^{n-1}  d_{u, v}x_{u,i}x_{v, (i+1)\pmod{n}} + P \left( \sum_{u=0}^{n-1} \left( \sum_{i=0}^{n-1} x_{ui}-1\right)^2 +\sum_{i=0}^{n-1} \left( \sum_{u=0}^{n-1} x_{ui}-1\right)^2 \right)$$

Users just have to provide the value of $P$ for a TSP class instance to generate QAOA [2] and XQAOA [5] circuits. 

It suffices to set $P > \max_{u,v} d_{u,v}$.

Below is the code to generate a QUBO with the penalty method, after which, we show various alternatives that are available for us to attempt the instance.


In [ ]:
penalty_qaoa_feasibility_dict = dict()
print("penalty, layer, tour, obj, prob_feasibility, result")
for penalty_pert in range(1,4):
    penalty  = dist_matrix.max() + penalty_pert
    TSP_with_penalty =  small_tsp.penalty_method(penalty)
    for layer in range(2, 5):
        gammas = np.random.rand(layer)
        betas = np.random.rand(layer)
        output = TSP_with_penalty.train_QAOA(gammas=gammas, betas=betas)
        output_freq = output[-1]
        cycle, obj, prob_feasibility, cauchy_dict = find_mode_from_result(output_freq)
        print(penalty, layer, cycle, obj, prob_feasibility, cauchy_dict)
        penalty_qaoa_feasibility_dict[(penalty, layer)] =  prob_feasibility

Another alternative that we can consider is via XQAOA approach that increases expressibility by having more parameters.  To do so, we just have to provide an additional parameter alphas to the train_QAOA method and the results are as follows:

In [ ]:
penalty_qaoa_feasibility_xqaoa_dict = dict()
print("penalty, layer, tour, obj, prob_feasibility, result")
for penalty_pert in range(1,4):
    penalty  = dist_matrix.max() + penalty_pert
    TSP_with_penalty =  small_tsp.penalty_method(penalty)
    for layer in range(2, 5):
        alphas = np.random.rand(layer)
        gammas = np.random.rand(layer)
        betas = np.random.rand(layer)
        output = TSP_with_penalty.train_QAOA(gammas=gammas, betas=betas, alphas=alphas)
        output_freq = output[-1]
        cycle, obj, prob_feasibility, cauchy_dict = find_mode_from_result(output_freq)
        print(penalty, layer, cycle, obj, prob_feasibility, cauchy_dict)
        penalty_qaoa_feasibility_xqaoa_dict[(penalty, layer)] =  prob_feasibility

Next, let's explore explore the performance using cVar loss function [6], we set the cVar parameter to be 0.25.

In [ ]:
penalty_qaoa_feasibility_cvar_dict = dict()
print("penalty, layer, tour, obj, prob_feasibility, result")
for penalty_pert in range(1,4):
    penalty  = dist_matrix.max() + penalty_pert
    TSP_with_penalty =  small_tsp.penalty_method(penalty)
    for layer in range(2, 5):
        alphas = np.random.rand(layer)
        gammas = np.random.rand(layer)
        betas = np.random.rand(layer)
        output = TSP_with_penalty.train_QAOA(gammas=gammas, betas=betas, regular_loss=False)
        output_freq = output[-1]
        cycle, obj, prob_feasibility, cauchy_dict = find_mode_from_result(output_freq)
        print(penalty, layer, cycle, obj, prob_feasibility, cauchy_dict)
        penalty_qaoa_feasibility_cvar_dict[(penalty, layer)] =  prob_feasibility

With our code, we can also adjust the number of iterations for the solver. 

In [ ]:
penalty_qaoa_feasibility_maxiter_dict = dict()
print("penalty, layer, tour, obj, prob_feasibility, result")
for penalty_pert in range(1,4):
    penalty  = dist_matrix.max() + penalty_pert
    TSP_with_penalty =  small_tsp.penalty_method(penalty)
    for layer in range(2, 5):
        alphas = np.random.rand(layer)
        gammas = np.random.rand(layer)
        betas = np.random.rand(layer)
        output = TSP_with_penalty.train_QAOA(gammas=gammas, betas=betas, maxiter=25)
        output_freq = output[-1]
        cycle, obj, prob_feasibility, cauchy_dict = find_mode_from_result(output_freq)
        print(penalty, layer, cycle, obj, prob_feasibility, cauchy_dict)
        penalty_qaoa_feasibility_cvar_dict[(penalty, layer)] =  prob_feasibility

We can also change a solver easily, in the following cell, we try the CMAES solver [7].

In [ ]:
penalty_qaoa_feasibility_cma_dict = dict()
print("penalty, layer, tour, obj, prob_feasibility, result")
for penalty_pert in range(1,4):
    penalty  = dist_matrix.max() + penalty_pert
    TSP_with_penalty =  small_tsp.penalty_method(penalty)
    for layer in range(2, 5):
        alphas = np.random.rand(layer)
        gammas = np.random.rand(layer)
        betas = np.random.rand(layer)
        output = TSP_with_penalty.train_QAOA(gammas=gammas, betas=betas, maxiter=25, method="cma")
        output_freq = output[-1]
        cycle, obj, prob_feasibility, cauchy_dict = find_mode_from_result(output_freq)
        print(penalty, layer, cycle, obj, prob_feasibility, cauchy_dict)
        penalty_qaoa_feasibility_cma_dict[(penalty, layer)] =  prob_feasibility

# Reference

1. Robinson, J. (1949). On the Hamiltonian game (a traveling salesman problem) (No. RANDRM303).
2. Farhi, E., Goldstone, J., & Gutmann, S. (2014). A quantum approximate optimization algorithm. arXiv preprint arXiv:1411.4028.
3. Lucas, A. (2014). Ising formulations of many NP problems. Frontiers in physics, 2, 5.
4. Hadfield, S., Wang, Z., O’gorman, B., Rieffel, E. G., Venturelli, D., & Biswas, R. (2019). From the quantum approximate optimization algorithm to a quantum alternating operator ansatz. Algorithms, 12(2), 34.
5. Vijendran, V., Das, A., Koh, D. E., Assad, S. M., & Lam, P. K. (2024). An expressive ansatz for low-depth quantum approximate optimisation. Quantum Science and Technology, 9(2), 025010.
6. Barkoutsos, P. K., Nannicini, G., Robert, A., Tavernelli, I., & Woerner, S. (2019). Improving variational quantum optimization using CVaR. arXiv preprint arXiv:1907.04769.
7. Auger, A., & Hansen, N. (2012, July). Tutorial CMA-ES: evolution strategies and covariance matrix adaptation. In Proceedings of the 14th annual conference companion on Genetic and evolutionary computation (pp. 827-848).